# Barzinga

In [ ]:
SEED = 213018

In [ ]:
#@title

import os
import shutil
from functools import reduce

from math import ceil

import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#@title

sns.set(palette=sns.color_palette("hls", 8))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

## Collecting Data

In [ ]:
DATASET_ZIP = '/content/drive/My Drive/Colab Notebooks/ml-notes/datasets/barzinga.zip'
DATASET = './ds/barzinga/images/barzinga'
LOGGING = '/content/drive/My Drive/Colab Notebooks/ml-notes/logs/barzinga'

BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)
INPUT_SHAPE = IMAGE_SIZE + (3,)

VALIDATION_SPLIT = 0.3

In [ ]:
import zipfile

with zipfile.ZipFile(DATASET_ZIP) as z:
    z.extractall('./ds/')

In [ ]:
x = tf.keras.preprocessing.image_dataset_from_directory(DATASET,
                                                        batch_size=BATCH_SIZE,
                                                        image_size=IMAGE_SIZE,
                                                        validation_split=0.3,
                                                        subset='training',
                                                        shuffle=True,
                                                        seed=SEED)

v = tf.keras.preprocessing.image_dataset_from_directory(DATASET,
                                                        batch_size=BATCH_SIZE,
                                                        image_size=IMAGE_SIZE,
                                                        validation_split=0.3,
                                                        subset='validation',
                                                        shuffle=True,
                                                        seed=SEED)

class_names = x.class_names

In [ ]:
#@title

plt.figure(figsize=(12, 12))

for images, labels in x.take(1):
    for i, image in enumerate(images):
        plt.subplot(ceil(len(images) / 6), 6, i + 1)
        plt.imshow(image.numpy().astype("uint8"))
        plt.axis("off")
plt.tight_layout()

## Augumentation Policy

In [ ]:
data_augmentation = tf.keras.Sequential(
    name='augmentation',
    layers=[
        tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
        tf.keras.layers.experimental.preprocessing.RandomTranslation(.1, .1),
        tf.keras.layers.experimental.preprocessing.RandomRotation(1.),
        tf.keras.layers.experimental.preprocessing.RandomZoom(.1),
        tf.keras.layers.experimental.preprocessing.RandomContrast(.25),
    ])

In [ ]:
#@title

from math import ceil

plt.figure(figsize=(12, 12))

for images, labels in x.take(1):
    images = data_augmentation(images).numpy()

    for i, image in enumerate(images):
        plt.subplot(ceil(len(images) / 6), 6, i + 1)
        plt.imshow(image.clip(0, 255).astype("uint8"))
        plt.axis("off")

plt.tight_layout()

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

xc = x.cache().prefetch(buffer_size=AUTOTUNE)
vc = v.cache().prefetch(buffer_size=AUTOTUNE)

## Defining Model

In [ ]:
mnetv2 = tf.keras.applications.MobileNetV2(weights='imagenet',
                                           include_top=False,
                                           input_shape=IMAGE_SIZE + (3,))

mnetv2.trainable = False

In [ ]:
from tensorflow.keras.layers import (GlobalAveragePooling2D,
                                     Dense)

def build_model(augmentation,
                preprocessing_fn,
                base_model,
                classes=10):
    x = tf.keras.Input(shape=INPUT_SHAPE, name='images')
    
    y = augmentation(x) if augmentation else x
    y = preprocessing_fn(y)
    y = base_model(y, training=False)

    y = GlobalAveragePooling2D(name='avg')(y)
    y = Dense(classes, name='predictions')(y)

    return tf.keras.Model(x, y)

barzinga = build_model(
    data_augmentation,
    tf.keras.applications.mobilenet_v2.preprocess_input,
    mnetv2,
    len(class_names)
)

tf.keras.utils.plot_model(barzinga, show_shapes=True, rankdir='LR')

## Training

In [ ]:
TRAINING = False

TRAINING_LOG = LOGGING + '/mnetv2'
TRAINING_WEIGHTS = TRAINING_LOG + '/weights.h5'

EPOCHS = 100
EPOCHS_FINE_TUNING = 100

In [ ]:
if TRAINING:
    shutil.rmtree(TRAINING_LOG, ignore_errors=True)

    barzinga.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy(),
                tf.keras.metrics.SparseTopKCategoricalAccuracy()])

    barzinga.fit(
        xc,
        epochs=EPOCHS,
        validation_data=vc,
        callbacks=[
            tf.keras.callbacks.TerminateOnNaN(),
            tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=10, verbose=1),
            tf.keras.callbacks.EarlyStopping(patience=30, verbose=1),
            tf.keras.callbacks.ModelCheckpoint(
                TRAINING_WEIGHTS,
                save_best_only=True,
                verbose=1),
            tf.keras.callbacks.TensorBoard(
                TRAINING_LOG + '/base',
                histogram_freq=1,
                write_images=True)
        ]);

### Fine-tuning

In [ ]:
mnetv2.trainable = True

if TRAINING:
    initial_epoch = len(barzinga.history.epoch)

    barzinga.compile(
        optimizer=tf.keras.optimizers.Adam(1e-5),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy(),
                tf.keras.metrics.SparseTopKCategoricalAccuracy()])

    barzinga.fit(
        xc,
        epochs=EPOCHS_FINE_TUNING + initial_epoch,
        validation_data=vc,
        initial_epoch=initial_epoch,
        callbacks=[
            tf.keras.callbacks.TerminateOnNaN(),
            tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3, verbose=1),
            tf.keras.callbacks.EarlyStopping(patience=10, verbose=1),
            tf.keras.callbacks.ModelCheckpoint(
                TRAINING_WEIGHTS,
                save_best_only=True,
                verbose=1),
            tf.keras.callbacks.TensorBoard(
                TRAINING_LOG + '/finetuning',
                histogram_freq=1,
                write_images=True)
        ]);

## Testing

In [ ]:
barzinga.load_weights(TRAINING_WEIGHTS)

In [ ]:
v = tf.keras.preprocessing.image_dataset_from_directory(DATASET,
                                                        batch_size=BATCH_SIZE,
                                                        image_size=IMAGE_SIZE,
                                                        # validation_split=0.3,
                                                        # subset='validation',
                                                        shuffle=False,
                                                        seed=SEED)

class_names = v.class_names

In [ ]:
y = []
p = []

for images, labels in v:
    y.append(labels)
    p.append(barzinga.predict_on_batch(images))

y = np.concatenate(y)
p = np.concatenate(p)

In [ ]:
class_names = np.asarray(class_names)

ps = tf.nn.softmax(p, axis=1).numpy()
l = np.argmax(ps, axis=1)

predictions = class_names[l]
true_labels = class_names[y]

### Fine-grained Classification Report

In [ ]:
from sklearn import metrics

print(metrics.classification_report(true_labels, predictions, labels=class_names))

In [ ]:
#@title

def confusion_matrix(y, p, labels):
    c = metrics.confusion_matrix(y, p, labels)
    r = c / (c.sum(axis=1, keepdims=True) + 1e-7)

    by_coherence = np.argsort(np.diag(r))[::-1]

    plt.figure(figsize=(16, 12))
    ax = sns.heatmap(r[:, by_coherence][by_coherence],
                     linewidths=.5, cmap='RdPu', cbar=False,
                     # annot=True, fmt='.0%',
                     xticklabels=labels[by_coherence],
                     yticklabels=labels[by_coherence])
    
confusion_matrix(true_labels, predictions, class_names)

In [ ]:
top_k = tf.nn.top_k(ps, k=5)

In [ ]:
top_k_tp = (top_k[1].numpy() == y.reshape(-1, 1)).astype(int)

print('top-5 accuracy:', top_k_tp.max(axis=1).mean().round(3))
print('top-5 confidence:', (top_k[0].numpy() * top_k_tp).max(axis=1).mean().round(3))

### Group Classification Report

In [ ]:
#@title

LABEL_GROUP = {
    'agtal-castanha': 'agtal',
    'amstel-larger': 'lata',
    'ana-maria-banana-chocolate': 'ana-maria',
    'ana-maria-coberta': 'ana-maria',
    'ana-maria-duplo-chocolate': 'ana-maria',
    'batatas-doces': 'batatas-doce',
    'batom': 'batom',
    'batom-branco': 'batom',
    'bauducco-duo': 'bauducco',
    'belvita': 'belvita',
    'capsula-0': 'capsula',
    'capsula-1': 'capsula',
    'capsula-2': 'capsula',
    'capsula-3': 'capsula',
    'capsula-4': 'capsula',
    'charge': 'charge',
    'cheetos': 'cheetos',
    'cheetos-assado': 'cheetos',
    'chikito': 'chikito',
    'club-social-integral': 'club-social',
    'club-social-original': 'club-social',
    'coca-plus-cafe': 'coca',
    'coca-zero': 'coca',
    'diamante-negro': 'diamante-negro',
    'eisenbahn-pilsen': 'lata',
    'eucalipto': 'eucalipto',
    'fanta': 'lata',
    'fanta-laranja': 'lata',
    'feel-good-cha-branco': 'feel-good',
    'feel-good-cha-branco-lichia': 'feel-good',
    'feel-good-cha-verde': 'feel-good',
    'feel-good-cha-verde-laranja': 'feel-good',
    'fini-tubes-m': 'fini',
    'fini-tubes-r': 'fini',
    'fini-tubes-uva': 'fini',
    'fini-tubes-v': 'fini',
    'flermel-goiabada': 'flermel',
    'flermel-pe-de-moleque': 'flermel',
    'flormel-abobora-coco': 'flermel',
    'flormel-chips-coco': 'flermel',
    'flormel-chips-coco-gengibre': 'flermel',
    'flormel-cocada': 'flermel',
    'flormel-crispy-grao-bico-ervas': 'flermel',
    'flormel-crispy-grao-bico-tomate-oregano': 'flermel',
    'flormel-doce-leite-coco': 'flermel',
    'flormel-doce-leite-nozes': 'flermel',
    'foda-se-o-que-e-isso': 'foda-se-o-que-e-isso',
    'formel-coco-abacaxi': 'flermel',
    'formel-doce-leite': 'flermel',
    'fruit-tella': 'fruit-tella',
    'gengibrinha': 'gengibrinha',
    'gengibrinha-aaaaaaaaaa': 'gengibrinha',
    'gengibrinha-hortela': 'gengibrinha',
    'gengibrinha-outra-sei-la': 'gengibrinha',
    'halls': 'halls',
    'heinken': 'lata',
    'ice-tea-leao': 'ice-tea',
    'kitkat': 'kitkat',
    'kitkat-white': 'kitkat',
    'laka-ao-leite': 'laka',
    'laka-branco': 'laka',
    'laka-oreo': 'laka',
    'lollo': 'lollo',
    'mandioquinha-chips': 'mandioquinha',
    'mems': 'mems',
    'mentos-rosa': 'mentos',
    'mix-batata-doces': 'mixed',
    'mixed-nuts-mel': 'mixed',
    'nativo': 'nativo',
    'nutry-aveia': 'nutry',
    'nutry-banana': 'nutry',
    'nutry-morango-chocolate': 'nutry',
    'remix-berry': 'remix',
    'remix-cacau': 'remix',
    'remix-frutas': 'remix',
    'schewpesepespespepses': 'lata',
    'sprite': 'lata',
    'talento': 'talento',
    'talento-branco': 'talento',
    'talento-castanhas-do-para': 'talento',
    'talento-diet': 'talento',
    'tictac': 'tictac',
    'toddy': 'toddy',
    'torcida-bacon': 'torcida',
    'torcida-pizza': 'tictac',
    'tribos': 'tribos',
    'um-doce-estranho-que-parece-uma-lombada': 'um-doce-estranho-que-parece-uma-lombada',
    'valle-goiaba': 'valle',
    'valle-manga': 'valle',
    'valle-pessego-maca': 'valle',
    'yoki-natural': 'yoki-natural',
}

In [ ]:
group_names = np.asarray(sorted(set(LABEL_GROUP.values())))
true_groups = [LABEL_GROUP[l] for l in true_labels]
predicted_groups = [LABEL_GROUP[l] for l in predictions]

In [ ]:
print(metrics.classification_report(true_groups, predicted_groups))

In [ ]:
confusion_matrix(true_groups, predicted_groups, group_names)

## Visualizing Activations

In [ ]:
@tf.function
def smooth_gradients(model, images, indices, num_samples=5, noise=1.):
    x = tf.repeat(images, num_samples, axis=0)
    x += tf.random.normal(x.shape, mean=0, stddev=noise)
    
    y = tf.repeat(indices, num_samples)

    with tf.GradientTape() as tape:
        tape.watch(x)
        logits = model(x)
        loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(y, logits)

    gs = tape.gradient(loss_, x)
    gs = tf.reshape(gs, (-1, num_samples, *gs.shape[1:]))
    
    return tf.reduce_mean(gs, axis=1)

@tf.function
def as_gray_mask(gs,
                 modify_gradients_pre=tf.abs):
    gs = modify_gradients_pre(gs) if modify_gradients_pre else gs
    gs = tf.reduce_mean(gs, axis=-1)

    return gs

In [ ]:
preprocessing_fn = tf.keras.applications.mobilenet_v2.preprocess_input
raw_input_model = tf.keras.Sequential(barzinga.layers[4:])

In [ ]:
for images, labels in xc:
    x = preprocessing_fn(images)
    gradients = smooth_gradients(raw_input_model, images, labels)
    masks = as_gray_mask(gradients)
    masks = tf.image.per_image_standardization(masks)

    break

In [ ]:
plt.figure(figsize=(16, 16))

for idx, (i, m) in enumerate(zip(images.numpy(), masks.numpy())):
    plt.subplot(8, 8, 2*idx+1)
    plt.imshow(i.astype(np.uint8))
    plt.axis('off')
    
    plt.subplot(8, 8, 2*idx+2)
    plt.imshow(m)
    plt.axis('off')

plt.tight_layout()